# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv into dataframe
cities_csv = "output_data/cities.csv"
city_df = pd.read_csv(cities_csv)
city_df.head()

,City,Country,Lat,Long,Cloudiness,Humidity,Max Temp,Wind Speed,Date
0,atuona,PF,-9.80,-139.03,0,75,78.31,21.39,1607615923
1,katherine,AU,-14.47,132.27,40,69,82.40,10.29,1607615923
2,hermanus,ZA,-34.42,19.23,15,69,69.01,5.99,1607615923
3,avarua,CK,-21.21,-159.78,21,83,78.80,8.05,1607615923
4,manokwari,ID,-0.87,134.08,74,76,80.67,1.43,1607615923


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Long"]]

In [5]:
# Create heat map of all cities based on humidity.

# Humidity values as floats
Humidity = city_df["Humidity"].astype(float)

# Map zoom and centering
coordinates = (20.12, 78.92)   # Centered on a random city in India
fig1 = gmaps.figure(center=coordinates, zoom_level=1.5)     # Zoomed to look better initially

# Create heat layer
heat_layer1 = gmaps.heatmap_layer(locations, weights=Humidity,dissipating=False, max_intensity=100,point_radius=1)

# Add layer
fig1.add_layer(heat_layer1)

# Display figure
fig1



Figure(layout=FigureLayout(height='420px'))

In [6]:
# Save gmap image as .png file
#Download button

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Choose the appx 10 cities with the most ideal temperature, wind speed, and cloudiness.
vac_city_df = city_df.copy()
vac_city_df = vac_city_df.dropna()
vac_city_df = vac_city_df[vac_city_df['Max Temp'] >= 68]
vac_city_df = vac_city_df[vac_city_df['Max Temp'] <= 77]  
vac_city_df = vac_city_df[vac_city_df['Wind Speed'] <= 10]
vac_city_df = vac_city_df[vac_city_df['Cloudiness'] == 0]

vac_city_df = vac_city_df.reset_index()
vac_city_df

,index,City,Country,Lat,Long,Cloudiness,Humidity,Max Temp,Wind Speed,Date
0,22,najran,SA,17.49,44.13,0,21,71.60,6.93,1607615927
1,66,guerrero negro,MX,17.67,-100.00,0,54,76.42,1.63,1607615935
2,159,mandal,IN,25.45,74.57,0,33,68.04,3.94,1607615954
3,232,pathein,MM,16.78,94.73,0,83,69.51,5.17,1607615968
4,238,narasannapeta,IN,18.42,84.05,0,75,70.83,2.01,1607615969
5,297,rajur,IN,20.12,78.92,0,52,70.09,2.59,1607615981
6,302,doha,QA,25.29,51.53,0,56,73.40,8.05,1607615982
7,309,banda aceh,IN,25.48,80.33,0,45,68.25,2.86,1607615983
8,386,rajgarh,IN,28.63,75.38,0,39,68.59,6.89,1607615998
9,488,paita,PE,-5.09,-81.11,0,69,77.00,8.90,1607616049


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create hotel_df with empty hotel name column
hotel_df = vac_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,City,Country,Lat,Long,Cloudiness,Humidity,Max Temp,Wind Speed,Date,Hotel Name
0,najran,SA,17.49,44.13,0,21,71.60,6.93,1607615927,NaN
1,guerrero negro,MX,17.67,-100.00,0,54,76.42,1.63,1607615935,NaN
2,mandal,IN,25.45,74.57,0,33,68.04,3.94,1607615954,NaN
3,pathein,MM,16.78,94.73,0,83,69.51,5.17,1607615968,NaN
4,narasannapeta,IN,18.42,84.05,0,75,70.83,2.01,1607615969,NaN
5,rajur,IN,20.12,78.92,0,52,70.09,2.59,1607615981,NaN
6,doha,QA,25.29,51.53,0,56,73.40,8.05,1607615982,NaN
7,banda aceh,IN,25.48,80.33,0,45,68.25,2.86,1607615983,NaN
8,rajgarh,IN,28.63,75.38,0,39,68.59,6.89,1607615998,NaN
9,paita,PE,-5.09,-81.11,0,69,77.00,8.90,1607616049,NaN


In [9]:
# Search GooglePlaces for hotels within 5000 meters
# List for hotel names
hotels = []

# Initialize counter
x = 0

# Loop to search based on city Lat and Long
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Lat'][x])+' , '+str(hotel_df['Long'][x])
    x+=1
    target_radius = 5000    # in meters
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

# Append dataframe with hotel names
hotel_df['Hotel Name'] = hotels

# Drop cities with no resulting hotel
hotel_df = hotel_df.dropna()

hotel_df

,City,Country,Lat,Long,Cloudiness,Humidity,Max Temp,Wind Speed,Date,Hotel Name
0,najran,SA,17.49,44.13,0,21,71.60,6.93,1607615927,محمد هادي ال فطيح
2,mandal,IN,25.45,74.57,0,33,68.04,3.94,1607615954,पीपी वाले हनुमान जी
3,pathein,MM,16.78,94.73,0,83,69.51,5.17,1607615968,Pathein Hotel
4,narasannapeta,IN,18.42,84.05,0,75,70.83,2.01,1607615969,G.lingaraju home
5,rajur,IN,20.12,78.92,0,52,70.09,2.59,1607615981,N D K Tee Point
6,doha,QA,25.29,51.53,0,56,73.40,8.05,1607615982,Doha Marriott Hotel
7,banda aceh,IN,25.48,80.33,0,45,68.25,2.86,1607615983,Hotel Comfort-Inn
8,rajgarh,IN,28.63,75.38,0,39,68.59,6.89,1607615998,Gangster Life Rajgarh Aala Ki
9,paita,PE,-5.09,-81.11,0,69,77.00,8.90,1607616049,Hotel Náutico de Paita
10,akyab,MM,20.15,92.90,0,74,71.69,7.52,1607616065,Hotel Memory


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Pin data
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [11]:
# Plot Ideal Weather Cities Heatmap

# Humidity values as floats
Humidity1 = hotel_df["Humidity"].astype(float)

# Map zoom and centering
coordinates = (20.12, 78.92)    # Centered on a random city in India
fig = gmaps.figure(center=coordinates, zoom_level=2.0)   # Zoomed for visually appealing presentation

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity1, dissipating=False, max_intensity=50, point_radius=1)

# Create marker layer
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Add layers
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [12]:
# Save gmap image as .png file
#fig.savefig('Vacation-Map.png')